In [10]:
#Zirui's ROI script
from becquerel import Spectrum
import numpy as np
import matplotlib.pyplot as plt

#Input: Spec, Bg, E_Peak, side_bands(have a default)
#Method: set_sidebands, get_counts

class ROI(object):
    def __init__ (self, spec, bg, e_peak):
        self.spec = spec
        self.bg = bg
        self.target_peak = e_peak
        self.delta_E = 5
        self.window = np.array([[-2, -1], [-0.5, 0.5], [1, 2]])

    def set_sideband (self, delta_e, window):
        if (np.array(window).ndim != 2 or len(window) != 3):
            print ("Wrong input dimension.")
            return

        self.delta_E = delta_e
        self.window = np.array(window)

    def get_counts (self):
        print(len(self.spec))
        print(len(self.bg))
        bgsub = self.spec - self.bg
        idx = (bgsub.energies_kev > self.target_peak+self.window[0,0]*self.delta_E)*(bgsub.energies_kev < self.target_peak+self.window[0,1]*self.delta_E)
        prev_bins = np.where(idx)
        idx = (bgsub.energies_kev > self.target_peak+self.window[1,0]*self.delta_E)*(bgsub.energies_kev < self.target_peak+self.window[1,1]*self.delta_E)
        curr_bins = np.where(idx)
        idx = (bgsub.energies_kev > self.target_peak+self.window[2,0]*self.delta_E)*(bgsub.energies_kev < self.target_peak+self.window[2,1]*self.delta_E)
        post_bins = np.where(idx)
        counts_1 = np.sum(bgsub.cps_vals[prev_bins[0][0]:prev_bins[0][-1]]) * self.spec.livetime
        counts_2 = np.sum(bgsub.cps_vals[post_bins[0][0]:post_bins[0][-1]]) * self.spec.livetime
        counts_target = np.sum(bgsub.cps_vals[curr_bins[0][0]:curr_bins[0][-1]]) * self.spec.livetime
        background = (counts_1 + counts_2)/2
        net_counts = counts_target - background

        return net_counts


In [11]:
#Austin's PF script
import becquerel as bq
from becquerel import Spectrum
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import math as m

#INPUT: source_energies, spectrum, background, branching_ratio

class PF(object):
    def __init__(self,source_energies,source_activities,spectrum,background,branching_ratio):
        self.source_energies = source_energies
        self.source_activities = source_activities
        self.spectrum = spectrum
        self.background = background
        self.branching_ratio = branching_ratio
    def Efficiency(self):
        def f_near(energy_array,energy): #finds index of closest energy in spectrum to that of the characteristic energy
            idx = np.abs(energy_array - energy).argmin()
            return idx
        def calibration(integrals): #calculate efficiencies
            efficiencies = []
            for x in range(0,len(source_activities)):
                iso = bq.tools.Isotope(source_isotopes[x])
                efficiency = integrals[x]/(source_activities[x]*branching_ratio[x])
                efficiencies = np.append(efficiencies,efficiency)
            return efficiencies
        spec = Spectrum.from_file(self.spectrum) #import spectrum data
        spec_counts = spec.counts_vals #spectrum counts
        bg = Spectrum.from_file(self.background) #import spectrum data
        bg_counts = bg.counts_vals #background counts
        sub_spec = spec - bg #background subtraction
        spec_energies = sub_spec.energies_kev #all energues
        spec_counts = spec_counts - bg_counts #all counts
        integrals = []
        for n in self.source_energies:
            fit = bq.core.fitting.FitterGaussErfLine(x=sub_spec.channels, y=sub_spec.cps_vals, y_unc=sub_spec.cps_uncs)
            idx = f_near(spec_energies,n)
            fit.set_roi(idx-100,idx+100)
            fit.fit()
            amp = fit.result.params['gauss_amp'].value
            mu = fit.result.params['gauss_mu'].value
            sigma =fit.result.params['gauss_sigma'].value
            def gaussian(x):
                return (spec.livetime * amp /(m.sqrt(2*m.pi)*sigma)) * m.exp(- ((x-mu)**2) / (2*sigma**2))
            integral = integrate.quad(gaussian, idx-100, idx+100)
            integrals = np.append(integrals,integral[0])
        efficiencies = calibration(integrals)
        return efficiencies

In [13]:
#Efficiency Master
#Define inputs
source_isotopes = np.array(['K-40']);
source_energies = np.array([1460]);
source_activities = np.array([1]);
spectrum = 'UCB087_Wild_King_Salmon_3.Spe'
background = 'UCB096_Backgorund_2_13_17.Spe'
branching_ratio = np.array([1]);

#Call scripts
roi = ROI(Spectrum.from_file(spectrum), Spectrum.from_file(background), source_energies)
roi_result = roi.get_counts()
peakfit = PF(source_energies,source_activities,spectrum,background,branching_ratio)
pf_result = peakfit.Efficiency()
print(roi_result, pf_result)

SpeFile: Reading file UCB087_Wild_King_Salmon_3.Spe
SpeFile: Reading file UCB096_Backgorund_2_13_17.Spe
16384
16384


/Users/austinshih/Documents/GitHub/becquerel/becquerel/core/spectrum.py:555: SpectrumWarning: Subtraction of counts-based specta, spectra have been converted to CPS
  'have been converted to CPS', SpectrumWarning)


SpeFile: Reading file UCB087_Wild_King_Salmon_3.Spe
SpeFile: Reading file UCB096_Backgorund_2_13_17.Spe
51548.5 [57966.7940058]
